# Grid 4x4

In [2]:
import csv
import os

import pandas as pd
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecMonitor
from sumo_rl import parallel_env
import supersuit as ss

from evaluate import evaluate

In [5]:
from stable_baselines3.common.utils import set_random_seed

TEST_NUM = 4
SEED = 23423  # default SUMO seed no.
set_random_seed(SEED)

## Create Environment and Instantiate Agent

In [9]:
from reward_functions import combined_reward

csv_dir = os.path.join("outputs","grid4x4",f"test_{TEST_NUM}")
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)

env_params = {
    "net_file": os.path.join("nets","grid4x4","grid4x4.net.xml"),
    "route_file": os.path.join("nets","grid4x4","grid4x4_1.rou.xml"),
    "num_seconds": 3600,
    "reward_fn": combined_reward,
    "sumo_seed": SEED,
}
congestion_reward = combined_reward.__defaults__[0].__name__
alpha = combined_reward.__defaults__[1]  # congestion component coefficient

env = parallel_env(**env_params)
# Maybe add frame-stacking here
env = ss.pettingzoo_env_to_vec_env_v1(env)

# Create env for evaluation
eval_env = VecMonitor(env)

# Create env for training
train_env = ss.concat_vec_envs_v1(env, num_vec_envs=8, base_class="stable_baselines3")
train_env = VecMonitor(train_env)

In [12]:
from hyperparams import custom

model = PPO(
    "MlpPolicy",
    train_env,
    tensorboard_log=os.path.join("logs","grid4x4"),
    verbose=1,
    # seed=SEED,
    **custom,
)

Using cuda device


## Evaluate Random Agent

In [45]:
# Evaluate untrained random agent
csv_path = os.path.join(csv_dir, "untrained.csv")
tb_log_dir = os.path.join("logs","grid4x4","eval_untrained")

reward_untrained, _ = evaluate(model, eval_env, csv_path, tb_log_dir, n_eval_episodes=1)

# Change made to SB3 > common > vec_env > vec_monitor.py > VecMonitor
# Line 76 - added extra item to self.venv.step_wait() return
# obs, rewards, dones, infos --> obs, rewards, dones, _, infos

print(reward_untrained)

-245990.18501207232
0.0


### Record Results

In [ ]:
untrained_csv = pd.read_csv(os.path.join(csv_dir, "untrained.csv"))
arrived_untrained = sum(untrained_csv.arrived_num)
pm_untrained = sum(untrained_csv.tyre_pm)
final_wait_untrained = untrained_csv.waiting_time.iat[-1]

data = [TEST_NUM, congestion_reward, alpha, 0,
        reward_untrained, arrived_untrained, pm_untrained, final_wait_untrained]

with open("grid_results.csv", "a", encoding="UTF8", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(data)

## Train Agent

In [46]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnNoModelImprovement

# Separate env for eval callback
callback_env = ss.concat_vec_envs_v1(env, num_vec_envs=1, base_class="stable_baselines3")
callback_env = VecMonitor(callback_env)

stop_train_callback = StopTrainingOnNoModelImprovement(max_no_improvement_evals=3, min_evals=5, verbose=1)
eval_callback = EvalCallback(callback_env, callback_after_eval=stop_train_callback, eval_freq=25000,
                             best_model_save_path=os.path.join("saved_models",f"grid4x4_{TEST_NUM}"))

TRAIN_STEPS = 1e6
model.learn(total_timesteps=TRAIN_STEPS, callback=eval_callback)

Logging to logs\grid4x4\PPO_3
-----------------------------------
| rollout/           |            |
|    ep_len_mean     | 720        |
|    ep_rew_mean     | -10200.519 |
| time/              |            |
|    fps             | 179        |
|    iterations      | 1          |
|    time_elapsed    | 729        |
|    total_timesteps | 131072     |
-----------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 720          |
|    ep_rew_mean          | -10065.127   |
| time/                   |              |
|    fps                  | 174          |
|    iterations           | 2            |
|    time_elapsed         | 1498         |
|    total_timesteps      | 262144       |
| train/                  |              |
|    approx_kl            | 0.0016639231 |
|    clip_fraction        | 0.0159       |
|    clip_range           | 0.1          |
|    entropy_loss         | -2.08        |
|    exp

In [47]:
# Evaluate trained agent
csv_path = os.path.join(csv_dir, "trained.csv")
tb_log_dir = os.path.join("logs","grid4x4",f"PPO_{TEST_NUM}","eval_trained")

reward_trained, _ = evaluate(model, eval_env, csv_path, tb_log_dir, n_eval_episodes=1)

print(reward_trained)

-143693.29015433788
0.0


In [48]:
# Move untrained.csv into numbered directory
import shutil

original = os.path.join("logs","grid4x4","eval_untrained")
target = os.path.join("logs","grid4x4",f"PPO_{TEST_NUM}","eval_untrained")

shutil.move(original, target)

'logs\\grid4x4\\PPO_3\\eval_untrained'

### Record Results

In [ ]:
trained_csv = pd.read_csv(os.path.join(csv_dir, "trained.csv"))
arrived_trained = sum(trained_csv.arrived_num)
pm_trained = sum(trained_csv.tyre_pm)
final_wait_trained = trained_csv.waiting_time.iat[-1]

data = [TEST_NUM, congestion_reward, alpha, TRAIN_STEPS,
        reward_trained, arrived_trained, pm_trained, final_wait_trained]

with open("grid4x4_results.csv", "a", encoding="UTF8", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(data)

### Save Model

In [49]:
model.save(os.path.join("saved_models",f"PPO_grid4x4_{TEST_NUM}"))

## Render Video

In [51]:
env_params.update({"use_gui": True, "render_mode": "human"})
render_env = parallel_env(**env_params)

# Maybe add frame-stacking here
render_env = ss.pettingzoo_env_to_vec_env_v1(render_env)
render_env = VecMonitor(render_env)

In [52]:
import pyautogui

obs = render_env.reset()

folder_path = os.path.join("renders","grid4x4",f"test_{TEST_NUM}")
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

max_time = render_env.unwrapped.par_env.unwrapped.env.sim_max_time
delta_time = render_env.unwrapped.par_env.unwrapped.env.delta_time
vid_length = int(max_time/delta_time)

for i in range(vid_length):
    actions, _ = model.predict(obs, deterministic=True)
    obs, rewards, dones, infos = render_env.step(actions)

    im = pyautogui.screenshot(
        os.path.join(folder_path,f"img{i}.jpg"),
        region=(0, 0, 2560, 1542)
    )

render_env.close()  # clean up

In [53]:
import subprocess

subprocess.run([
    "ffmpeg", "-y", "-r", "30", "-i",
    f"renders\\grid4x4\\test_{TEST_NUM}\\img%d.jpg",
    f"videos\\grid4x4_{TEST_NUM}.mp4"
])

CompletedProcess(args=['ffmpeg', '-y', '-r', '5', '-i', 'renders\\grid4x4\\test_3\\img%d.jpg', 'videos\\grid4x4_3.mp4'], returncode=0)